In [1]:
import pandas as pd

In [2]:
meta_dataset = pd.read_csv("./performance_meta_dataset.csv", index_col=0)
single_sample_algs = [
    "TopPop", 
    "GlobalEffects", 
    "Random",
    "SlopeOne",
]
min_samples = 30

keep_rows = (meta_dataset["num_samples"] >= min_samples) | meta_dataset["alg_name"].isin(single_sample_algs)
meta_dataset = meta_dataset.loc[keep_rows, :]

metafeats_fn = "../RecSys2019_DeepLearning_Evaluation/Metafeatures/Metafeatures.csv"
metafeats = pd.read_csv(metafeats_fn)
join_cols = ["dataset_name", "split_name"]
metafeats.columns = ["f__{}".format(col) if col not in join_cols else col for col in metafeats.columns]
del metafeats["split_name"]
metafeats = meta_dataset.merge(metafeats, on="dataset_name", how='left')
metafeats

,dataset_name,alg_name,min_test_metric_ARHR_ALL_HITS_cut_1,max_test_metric_ARHR_ALL_HITS_cut_1,min_test_metric_ARHR_ALL_HITS_cut_10,max_test_metric_ARHR_ALL_HITS_cut_10,min_test_metric_ARHR_ALL_HITS_cut_15,max_test_metric_ARHR_ALL_HITS_cut_15,min_test_metric_ARHR_ALL_HITS_cut_2,max_test_metric_ARHR_ALL_HITS_cut_2,...,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_mean,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_median,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_min,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_mode,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_skewness,f__dist_feature__kind_user__pre_agg_func_sum__agg_func_std,f__num_interactions,f__num_items,f__num_users,f__sparsity
0,AnimeReader,EASE_R_Recommender,0.000000,0.034277,0.000442,0.059715,0.000563,0.062664,0.000079,0.043217,...,676.747371,337.0,-10225.0,10.0,3.895959,1021.199357,7669090,11200,69521,0.990151
1,AnimeReader,GlobalEffects,0.000187,0.000187,0.004560,0.004560,0.005064,0.005064,0.000245,0.000245,...,676.747371,337.0,-10225.0,10.0,3.895959,1021.199357,7669090,11200,69521,0.990151
2,AnimeReader,ItemKNN,0.000000,0.043915,0.000002,0.070494,0.000004,0.073168,0.000000,0.054840,...,676.747371,337.0,-10225.0,10.0,3.895959,1021.199357,7669090,11200,69521,0.990151
3,AnimeReader,MatrixFactorization_BPR_Cython,0.000029,0.013248,0.000192,0.032227,0.000222,0.034679,0.000072,0.017613,...,676.747371,337.0,-10225.0,10.0,3.895959,1021.199357,7669090,11200,69521,0.990151
4,AnimeReader,P3alphaRecommender,0.000000,0.026841,0.000000,0.047001,0.000000,0.048477,0.000000,0.034572,...,676.747371,337.0,-10225.0,10.0,3.895959,1021.199357,7669090,11200,69521,0.990151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,WikilensReader,SLIMElasticNetRecommender,0.000000,0.040000,0.000000,0.060330,0.000000,0.065001,0.000000,0.045455,...,331.532727,100.5,0.5,5.0,4.232346,644.482503,26316,5111,275,0.981277
249,WikilensReader,SLIM_BPR_Cython,0.000000,0.021818,0.013332,0.036427,0.019057,0.037529,0.003636,0.023636,...,331.532727,100.5,0.5,5.0,4.232346,644.482503,26316,5111,275,0.981277
250,WikilensReader,SlopeOne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,331.532727,100.5,0.5,5.0,4.232346,644.482503,26316,5111,275,0.981277
251,WikilensReader,TopPop,0.007273,0.007273,0.018514,0.018514,0.020232,0.020232,0.007273,0.007273,...,331.532727,100.5,0.5,5.0,4.232346,644.482503,26316,5111,275,0.981277


In [3]:
list(meta_dataset.columns)

['dataset_name',
 'alg_name',
 'min_test_metric_ARHR_ALL_HITS_cut_1',
 'max_test_metric_ARHR_ALL_HITS_cut_1',
 'min_test_metric_ARHR_ALL_HITS_cut_10',
 'max_test_metric_ARHR_ALL_HITS_cut_10',
 'min_test_metric_ARHR_ALL_HITS_cut_15',
 'max_test_metric_ARHR_ALL_HITS_cut_15',
 'min_test_metric_ARHR_ALL_HITS_cut_2',
 'max_test_metric_ARHR_ALL_HITS_cut_2',
 'min_test_metric_ARHR_ALL_HITS_cut_20',
 'max_test_metric_ARHR_ALL_HITS_cut_20',
 'min_test_metric_ARHR_ALL_HITS_cut_3',
 'max_test_metric_ARHR_ALL_HITS_cut_3',
 'min_test_metric_ARHR_ALL_HITS_cut_30',
 'max_test_metric_ARHR_ALL_HITS_cut_30',
 'min_test_metric_ARHR_ALL_HITS_cut_4',
 'max_test_metric_ARHR_ALL_HITS_cut_4',
 'min_test_metric_ARHR_ALL_HITS_cut_40',
 'max_test_metric_ARHR_ALL_HITS_cut_40',
 'min_test_metric_ARHR_ALL_HITS_cut_5',
 'max_test_metric_ARHR_ALL_HITS_cut_5',
 'min_test_metric_ARHR_ALL_HITS_cut_50',
 'max_test_metric_ARHR_ALL_HITS_cut_50',
 'min_test_metric_ARHR_ALL_HITS_cut_6',
 'max_test_metric_ARHR_ALL_HITS_cut_6'

In [4]:
# Algorithm selection

def rank_algorithms(test_datasets, metric_name):
    """Compute algorithm ranks for each dataset"""
    # Sanity check to prevent leakage
    for test_dataset in test_datasets:
        assert test_dataset in meta_dataset['dataset_name'].values
    filtered_dataset = meta_dataset[~meta_dataset['dataset_name'].isin(test_datasets)]
    
    all_ranks = []
    for dataset_name, dataset_performance in filtered_dataset.groupby("dataset_name"):
        dataset_performance["rank"] = dataset_performance["max_test_metric_" + metric_name].rank(method='min', ascending=False)
        dataset_performance.set_index("alg_name", inplace=True)
        dataset_performance = dataset_performance[["rank"]]
        dataset_performance = dataset_performance.rename(columns={"rank": dataset_name})
        all_ranks.append(dataset_performance)
        
    ranked_algs = pd.concat(all_ranks, axis=1)
    return ranked_algs

def select_algs(test_datasets, metric_name, num_algs=10):
    """Select num_algs algorithm with best mean rank"""
    return list(ranked_algs.T.mean().sort_values().iloc[:num_algs].index)

In [5]:
# Metafeature selection

def compute_feature_corrs(test_datasets, metric_name, selected_algs):
    """Compute correlation between each metafeature and the desired metric for all selected algorithms.
    Dataframe result is num_features x num_algorithms."""
    all_features = [col for col in metafeats.columns if col.startswith("f__")]
    # Sanity check to prevent leakage
    for test_dataset in test_datasets:
        assert test_dataset in metafeats['dataset_name'].values
    filtered_metafeats = metafeats[~metafeats['dataset_name'].isin(test_datasets)]
    
    all_cors = []
    for alg in selected_algs:
        filtered_results = filtered_metafeats.loc[(filtered_metafeats["alg_name"] == alg)]
        alg_cors = filtered_results[all_features].corrwith(filtered_results["max_test_metric_" + metric_name],
                                                          method="spearman")
        alg_cors.name = alg
        all_cors.append(alg_cors)
    all_cors = pd.concat(all_cors, axis=1).abs()
    return all_cors

def select_features(test_datasets, metric_name, selected_algs, num_feats=10):
    """Select num_feats features. Greedy scheme. At each step, we compute the best correlations
    across all metafeatures for each algorithm so far. We add whichever metafeature can obtain the maximum
    improvement across any single one of the best correlations for the selected algorithms."""
    all_cors = compute_feature_corrs(test_datasets, metric_name, selected_algs)
    
    selected_feats = [all_cors.max(axis=1).idxmax()]
    
    while len(selected_feats) < num_feats:
        # Current best correlations
        current_best_cors = all_cors.loc[selected_feats].max(axis=0)
        # Pick whichever feature results in the highest maximum improvement on the current best correlations
        selected_feats.append((
            all_cors.loc[~all_cors.index.isin(selected_feats)] - current_best_cors)
            .max(axis=1)
            .idxmax())
    return selected_feats

In [6]:
# Assume some split
test_datasets = ["AnimeReader", "CiaoDVDReader"]
# Assume some metric
metric_name = "ARHR_ALL_HITS_cut_1"

In [7]:
ranked_algs = rank_algorithms(test_datasets, metric_name)
ranked_algs

,BookCrossingReader,DatingReader,EpinionsReader,FilmTrustReader,FrappeReader,GowallaReader,Jester2Reader,LastFMReader,MarketBiasAmazonReader,MarketBiasModClothReader,MovieTweetingsReader,Movielens100KReader,Movielens10MReader,Movielens1MReader,Movielens20MReader,MovielensHetrec2011Reader,NetflixPrizeReader,RecipesReader,WikilensReader
alg_name,,,,,,,,,,,,,,,,,,,
GlobalEffects,6.0,3.0,8.0,15.0,15.0,3.0,12.0,12.0,12.0,15.0,7.0,15.0,9.0,12.0,5.0,12.0,3.0,6.0,13.0
ItemKNN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
MatrixFactorization_BPR_Cython,4.0,NaN,6.0,8.0,14.0,NaN,5.0,5.0,11.0,13.0,5.0,12.0,6.0,8.0,NaN,7.0,NaN,4.0,9.0
PureSVDRecommender,2.0,NaN,3.0,12.0,10.0,NaN,7.0,10.0,7.0,11.0,4.0,9.0,7.0,9.0,3.0,10.0,NaN,3.0,8.0
Random,7.0,5.0,9.0,15.0,15.0,3.0,13.0,12.0,16.0,16.0,9.0,17.0,11.0,13.0,7.0,13.0,4.0,7.0,15.0
TopPop,3.0,3.0,4.0,8.0,11.0,2.0,2.0,9.0,10.0,7.0,6.0,14.0,8.0,10.0,4.0,11.0,2.0,2.0,13.0
UserKNN,4.0,2.0,7.0,11.0,7.0,1.0,10.0,5.0,14.0,14.0,8.0,12.0,10.0,10.0,6.0,8.0,NaN,5.0,6.0
P3alphaRecommender,NaN,NaN,2.0,4.0,3.0,NaN,8.0,3.0,1.0,2.0,3.0,5.0,5.0,6.0,2.0,5.0,NaN,NaN,3.0
RP3betaRecommender,NaN,NaN,5.0,2.0,5.0,NaN,9.0,4.0,6.0,6.0,2.0,2.0,3.0,2.0,NaN,4.0,NaN,NaN,5.0


In [8]:
selected_algs = select_algs(test_datasets, metric_name)
selected_algs

['ItemKNN',
 'SLIM_BPR_Cython',
 'P3alphaRecommender',
 'EASE_R_Recommender',
 'SLIMElasticNetRecommender',
 'RP3betaRecommender',
 'TopPop',
 'IALSRecommender',
 'PureSVDRecommender',
 'UserKNN']

In [9]:
feature_corrs = compute_feature_corrs(test_datasets, metric_name, selected_algs)
feature_corrs

,ItemKNN,SLIM_BPR_Cython,P3alphaRecommender,EASE_R_Recommender,SLIMElasticNetRecommender,RP3betaRecommender,TopPop,IALSRecommender,PureSVDRecommender,UserKNN
f__dist_feature__kind_item__pre_agg_func_count__agg_func_Gini,0.056760,0.081818,0.125275,0.006061,0.151515,0.318681,0.014035,NaN,0.004415,0.209494
f__dist_feature__kind_item__pre_agg_func_count__agg_func_kurtosis:,0.475748,0.072727,0.024176,0.163636,0.187879,0.082418,0.603509,NaN,0.487123,0.504644
f__dist_feature__kind_item__pre_agg_func_count__agg_func_max,0.114551,0.227273,0.432967,0.054545,0.103030,0.137363,0.250877,NaN,0.104489,0.182663
f__dist_feature__kind_item__pre_agg_func_count__agg_func_mean,0.244582,0.036364,0.301099,0.175758,0.054545,0.016484,0.270175,NaN,0.261957,0.261094
f__dist_feature__kind_item__pre_agg_func_count__agg_func_median,0.234207,0.004598,0.285749,0.195126,0.012309,0.028017,0.275044,NaN,0.298172,0.305653
...,...,...,...,...,...,...,...,...,...,...
f__dist_feature__kind_user__pre_agg_func_sum__agg_func_std,0.238390,0.272727,0.265934,0.406061,0.587879,0.263736,0.443860,NaN,0.598970,0.036120
f__num_interactions,0.318885,0.045455,0.586813,0.321212,0.381818,0.302198,0.496491,NaN,0.384106,0.308566
f__num_items,0.628483,0.300000,0.248352,0.321212,0.587879,0.247253,0.887719,NaN,0.734364,0.607843
f__num_users,0.403509,0.027273,0.481319,0.212121,0.309091,0.236264,0.468421,NaN,0.269316,0.543860


In [10]:
selected_feats = select_features(test_datasets, metric_name, selected_algs)
selected_feats

['f__num_items',
 'f__dist_feature__kind_user__pre_agg_func_sum__agg_func_mode',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_max',
 'f__dist_feature__kind_user__pre_agg_func_count__agg_func_kurtosis:',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_kurtosis:',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_mode',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_mean',
 'f__dist_feature__kind_item__pre_agg_func_mean__agg_func_median',
 'f__dist_feature__kind_user__pre_agg_func_sum__agg_func_max',
 'f__dist_feature__kind_user__pre_agg_func_sum__agg_func_kurtosis:']

In [11]:
ranked_algs.T.mean().sort_values().iloc[:10]

alg_name
ItemKNN                      1.277778
SLIM_BPR_Cython              3.636364
P3alphaRecommender           3.714286
EASE_R_Recommender           3.900000
SLIMElasticNetRecommender    4.100000
RP3betaRecommender           4.230769
TopPop                       6.789474
IALSRecommender              7.000000
PureSVDRecommender           7.187500
UserKNN                      7.777778
dtype: float64